In [ ]:
!pip install tf-keras-vis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.1/52.1 kB 576.7 kB/s eta 0:00:00


In [ ]:
## load package
%reload_ext autoreload
%autoreload 2

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tf_keras_vis.utils import num_of_gpus
import matplotlib.pyplot as plt
%matplotlib inline
from keras.models import Sequential
from sklearn.model_selection import train_test_split
_, gpus = num_of_gpus()
print('Tensorflow recognized {} GPUs'.format(gpus))

Tensorflow recognized 0 GPUs


In [ ]:
from tensorflow.keras.applications import EfficientNetV2B0 as Model

model = Model(weights='imagenet', include_top=True)
model.compile(
        loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(lr=0.01), metrics=["accuracy"]
    )
model.summary()

29403144/29403144 [==============================] - 0s 0us/step


Model: "efficientnetv2-b0"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 rescaling (Rescaling)       (None, 224, 224, 3)          0         ['input_1[0][0]']             
                                                                                                  
 normalization (Normalizati  (None, 224, 224, 3)          0         ['rescaling[0][0]']           
 on)                                                                                              
                                                                                                  
 stem_conv (Conv2D)          (None, 112, 112, 32)         864       ['normalizatio

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
FGSM_image_path = "/content/drive/MyDrive/Subset_1-40_81-120/Adversarial_Examples/FGSM"
FGSM_LIME_path = "/content/drive/MyDrive/Subset_1-40_81-120/FGSM_Adversarial_Explanation/LIME/"

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from keras.models import Sequential
from sklearn.model_selection import train_test_split


In [ ]:
from PIL import Image

def change_size_for_model(image):
    image = Image.fromarray(image)
    resized_image = image.resize((224, 224))
    resized_array = np.array(resized_image)
    return resized_array.astype(np.uint8)

def change_size_for_original(image, original_shape):
    image = Image.fromarray(image)
    resized_image = image.resize(original_shape) #resize image
    resized_array = np.array(resized_image)
    return resized_array.astype(np.uint8)

In [ ]:
start_idx =80
end_idx = 121

In [ ]:
list_of_dir =list()
for i in range(start_idx,end_idx):
  list_of_dir.append(str(i))

In [ ]:
from tf_keras_vis.utils.model_modifiers import ReplaceToLinear, ModelModifier

replace2linear = ReplaceToLinear()
def model_modifier_function(cloned_model):
    cloned_model.layers[-1].activation = tf.keras.activations.linear

In [ ]:
from tf_keras_vis.utils.scores import CategoricalScore

In [ ]:
pip install lime

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from lime import lime_image
from skimage.segmentation import mark_boundaries
from IPython.display import clear_output
from matplotlib import cm

In [ ]:
def predict_fn(images):
    return model.predict(images)


explainer = lime_image.LimeImageExplainer()

In [ ]:

# from tf_keras_vis.gradcam import Gradcam

# gradcam = Gradcam(model,
#                   model_modifier=model_modifier_function,
#                   clone=True)
for j in range(len(list_of_dir)):
# for j in range(3):
  try:
    list_of_names = os.listdir(FGSM_image_path+'/'+str(list_of_dir[j]))
    list_of_names.sort()
    original_image_array = list()

    for img_name in list_of_names:
      # Load the image
      image = Image.open(FGSM_image_path+'/'+str(list_of_dir[j])+'/'+img_name)

      # Convert the image to a numpy array
      original_image_array.append(np.array(image))

    resized_image_array = list()
    for img in original_image_array:
      resized_image_array.append(change_size_for_model(img))

    resized_image_array = np.array(resized_image_array)

    ################
    # Your code



    result = model.predict(resized_image_array)
    # saliency_map = list()
    LIME_attention_mask_array = list()
    for i in range(resized_image_array.shape[0]):
      X = resized_image_array[i]
      # print([np.argmax(result[i])])


      explanation = explainer.explain_instance(X, predict_fn, top_labels=1, num_samples=1000)
      temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=True, num_features=5, hide_rest=False)
      # lime_e_img = mark_boundaries(temp/1 + 0.5, mask)

      LIME_attention_mask_array.append(mask)




      # score = CategoricalScore(np.argmax(result[i]))
      # # print(np.argmax(result[i]))

      # cam = gradcam(score,
      #           X.astype(np.float32),
      #           penultimate_layer=-1)
      # # heatmap = np.uint8(cm.jet(cam[0])* 255)
      # # plt.figure()
      # # plt.imshow(X, cmap='gray')
      # # plt.imshow(heatmap, cmap='jet', alpha=0.5)
      # # plt.show()
      # saliency_map.append(cam)
      # # plt.imshow(saliency_map[i].reshape(224,224), cmap='gray')
      # # plt.show()
      # # plt.pause(1)

    ################

    folder_for_exp = FGSM_LIME_path + str(list_of_dir[j])
    if not os.path.exists(folder_for_exp):
        # The folder does not exist, so create it
        os.makedirs(folder_for_exp, exist_ok=True)
        print(f"Folder '{folder_for_exp}' was created.")
    else:
        # The folder already exists
        print(f"Folder '{folder_for_exp}' already exists.")

    for i in range(len(LIME_attention_mask_array)):
      a = LIME_attention_mask_array[i].reshape(224,224)
      #print(a.shape)
      # plt.figure()
      # plt.hist(a.flatten(), bins=256, range=(0, 255))
      # mod = np.percentile(a.flatten(),85)
      # # print(mod)
      # a[np.where(a >= mod)] = 255
      # a[np.where(a < mod)] = 0

      X = resized_image_array[i]

      # plt.figure()
      # plt.imshow(X, cmap='gray')
      # # # plt.imshow(heatmap, cmap='jet', alpha=0.5)

      # plt.imshow(a, cmap='binary_r', alpha = 0.5)
      # plt.show()
      # plt.pause(1)
      plt.imsave(folder_for_exp+'/' +str(list_of_names[i]), a, cmap='binary_r')
  except:
     pass

1/1 [==============================] - 0s 139ms/step


  0%|          | 0/1000 [00:00<?, ?it/s]

1/1 [==============================] - 1s 785ms/step


  0%|          | 0/1000 [00:00<?, ?it/s]

1/1 [==============================] - 0s 463ms/step
Folder '/content/drive/MyDrive/Subset_1-40_81-120/FGSM_Adversarial_Explanation/LIME/81' was created.
1/1 [==============================] - 0s 168ms/step


...

1/1 [==============================] - 0s 468ms/step


  0%|          | 0/1000 [00:00<?, ?it/s]

1/1 [==============================] - 0s 462ms/step
Folder '/content/drive/MyDrive/Subset_1-40_81-120/FGSM_Adversarial_Explanation/LIME/117' was created.
1/1 [==============================] - 0s 97ms/step


  0%|          | 0/1000 [00:00<?, ?it/s]

1/1 [==============================] - 0s 493ms/step
Folder '/content/drive/MyDrive/Subset_1-40_81-120/FGSM_Adversarial_Explanation/LIME/118' was created.
